In [ ]:
import pandas as pd
import glob

files = sorted(glob.glob('run_ca*.csv'))
df = pd.concat((pd.read_csv(f) for f in files), ignore_index=True)

print(f"Loaded {len(files)} files, total rows = {len(df)}")
df.head()


In [ ]:
# Compute per‐episode metrics
episode_stats = []
# group by run (to handle multiple runs), row_length and episode_index
for (_, row_len, ep), grp in df.groupby(['run_id', 'row_length', 'episode_index']):
    steps = len(grp)  # one row per timestep
    mean_time = grp['time_delta_ms'].mean()
    # compute switch rate
    actions = grp.sort_values('step_count')['action_taken'].tolist()
    if len(actions) > 1:
        switches = sum(1 for i in range(1,len(actions)) if actions[i] != actions[i-1])
        switch_rate = switches / (len(actions)-1)
    else:
        switch_rate = 0.0
    episode_stats.append({
        'row_length':   row_len,
        'steps':        steps,
        'time_ms':      mean_time,
        'switch_rate':  switch_rate
    })
eps_df = pd.DataFrame(episode_stats)

# Aggregate per row_length
summary = eps_df.groupby('row_length').agg(
    mean_steps   = ('steps',       'mean'),
    std_steps    = ('steps',       'std'),
    mean_time    = ('time_ms',     'mean'),
    std_time     = ('time_ms',     'std'),
    mean_switch  = ('switch_rate', 'mean'),
    std_switch   = ('switch_rate', 'std')
).reset_index()

In [ ]:
from matplotlib import pyplot as plt
# Avg Steps Survived vs. Row Length
plt.figure(figsize=(6, 4))
plt.plot(
    summary['row_length'],
    summary['mean_steps'],
    '-o'
)
plt.xlabel('CA Row Length (cells)')
plt.ylabel('Avg Steps')
plt.xticks(summary['row_length'])
plt.title('Steps Survived vs. Row Length')

# Add value labels to each point
for x, y in zip(summary['row_length'], summary['mean_steps']):
    plt.annotate(
        f"{y:.1f}",
        (x, y),
        textcoords="offset points",
        xytext=(0, 7),
        ha='center',
        fontsize=10,
        bbox=dict(boxstyle="round,pad=0.2", fc="white", ec="none", alpha=0.7)
    )

plt.tight_layout()
plt.show()

# Avg Decision Time vs. Row Length
plt.figure(figsize=(6, 4))
plt.plot(
    summary['row_length'],
    summary['mean_time'],
    '-s'
)
plt.xlabel('CA Row Length (cells)')
plt.ylabel('Avg Decision Time (ms)')
plt.xticks(summary['row_length'])
plt.title('Decision Time vs. Row Length')

# Add value labels to each point
for x, y in zip(summary['row_length'], summary['mean_time']):
    plt.annotate(
        f"{y:.1f}",
        (x, y),
        textcoords="offset points",
        xytext=(0, 7),
        ha='center',
        fontsize=10,
        bbox=dict(boxstyle="round,pad=0.2", fc="white", ec="none", alpha=0.7)
    )

plt.tight_layout()
plt.show()

In [ ]:

# Stability (Action‐Switch Rate) vs. Row Length
plt.figure(figsize=(6,4))
plt.bar(summary['row_length'], summary['mean_switch'],
        yerr=summary['std_switch'], capsize=5, color='gray', alpha=0.7)
plt.xlabel('CA Row Length (cells)')
plt.ylabel('Avg Action‐Switch Rate')
plt.xticks(summary['row_length'])
plt.title('Controller Stability vs. Row Length')
plt.tight_layout()
plt.show()
